# Análisis de las señales oblicuas usando inferencia bayesiana

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import interpolate
import seaborn as sns
sns.set()
import pandas as pd
import pickle

import theano.tensor as TT
import time
import random

import pymc3 as pm
import arviz as az

import sys
#import winsound

In [ ]:
%run Bayes3.ipynb
%run Pinel.ipynb
%run R_fresnel.ipynb

In [ ]:
## Parámetros globales de la antena

Fs = 23.328e9 #Frecuencia de muestreo [Hz] 
Ts = 1/Fs
BW = 1.5e9 

corte = 150
Nfft= 1024 # nro de frecs -> conviene que sea multiplo de 2 para calcular la FFT de manera eficiente. 
#Tiene que ser mayor a la longitud de la señal (285 para 2m, 439 para 3m)

portadora = input('Fc7 , Fc8')
mediciones = ['06_09','07_15','08_22']
dia = input('dia de medicion? (jun=1, jul=2, ag=3)')
dia = mediciones[int(dia)]

if portadora == 'Fc8':
    fc = 8.748e9
    print('fc =',str(fc/1e9) + 'GHz')
    f_pos = np.load('mediciones/f.npy',mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
    
    if dia == '06_09':
        altura = 1
        tita = input('titas posibles: 5, 10, 15, 20, 25, 30, 40, 50')
    elif dia == '07_15':
        altura = 0
        tita = input('titas posibles: 10, 20, 30, 40')
    else:
        altura = 1
        tita = input('titas posibles: 10, 20')
            
    with open('mediciones/altura_'+altura+'.pkl','rb') as Q:
        med = pickle.load(Q)
    Scal = med.get('S1_'+dia)
    with open('mediciones/Z2'+dia+'.pkl','rb') as QQ:
        Z2 = pickle.load(QQ)
    Star = Z2.get('Z2_'+tita)

theta = np.radians(int(tita))
else:
    fc = 7.290e9
    print('fc =',str(fc/1e9) + 'GHz')
    print('las mediciones para Fc7 no estan procesadas')

print('dim f_pos:',np.shape(f_pos))
print('dim S1:',np.shape(Scal))
print('dim Z2:',np.shape(Star))
print('tita:',str(theta)) 

In [ ]:
## Mediciones con el hydra
if Y == '0':
    # mayo
    ep1_obs = (20.731 + 1j*8.688) - 1j #2cm
    ep2_obs = (23.299 + 1j*9.799) - 1j #6,5cm
    ep3_obs = (25.813+10.474j) - 1j #12cm 
    
elif Y == '1':
    # junio
    ep1_obs = (11.496 +1j*4.028) - 1j #2,5cm
    ep2_obs = (13.894 + 1j*5.12) - 1j #7cm
    ep3_obs = (22.823 + 1j*18.73) - 1j #13cm

elif Y == '2':
    # julio
    ep1_obs = (8.129 +1j*4.91) -1j #2cm
    ep2_obs = (10.302 + 1j*6.12) -1j #7cm
    ep3_obs = (11.268 + 1j*6.695) -1j#14cm

else:
    # agosto
    ep1_obs = 7.6 + 1j*4.36 #2cm
    ep2_obs = 7.54 + 1j*3.94 #9cm
    ep3_obs = 'error' #no se medio otra humedad

# Ajustes con el esquema bayesiano:

In [ ]:
# %%capture cap --no-stderr
NLayers = 1
print('-------------------')
print('Incidencia en un dielectrico sin rugosidad')
print('-------------------')
print()
print('Datos:')
print('Dia de medicion:', dia)
print('angulo:', tita)
print('Altura:', altura)
name = 'lisa_'+dia+'-T'+tita
print()
print()
print('-------------------')
print('Corro bayes')
print('-------------------')

t0 = time.time()
# nro de muestras que va a utilizar el método
noise_level = 2.25e-2
print('Ruido : ',noise_level)
muestras = 5000
print('Muestras : ',muestras)
# error de la medición
sigmas = noise_level*np.ones((1,len(Star)))
print()


# inferencia bayesiana
traza_sim_1layer = modelo(Star,muestras,sigmas,NLayers,False)
t1 = time.time()
print('Tiempo insumido - Inf. Bayesiana: '+str(round(t1-t0,2))+' seg', '('+str(round((t1-t0)/60,2))+' min)')


t2 = time.time()
print('Graficando con un solo dielectrico ...')

ep1_r_mean = round(np.mean(traza_sim_1layer['ep1_r']),3)
ep1_r_std = round(np.std(traza_sim_1layer['ep1_r']),3)

ep1_i_mean = round(np.mean(traza_sim_1layer['ep1_i']),3)
ep1_i_std = round(np.std(traza_sim_1layer['ep1_i']),3)

plotear = np.stack((traza_sim_1layer['ep1_r'], traza_sim_1layer['ep1_i'])).T 
dfplotear = pd.DataFrame(plotear, columns = ['ep1_r', 'ep1_i'])


g = sns.PairGrid(dfplotear)
g.fig.suptitle('Medicion sintetica con un solo medio material liso', y=1.01, fontsize = 'large')
g.map_upper(sns.histplot)
# g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True, color='.5')
# g.axes[0,0].axvline(np.real(ep1_obs), color='r', linestyle='solid', linewidth=0.5)
# g.axes[1,0].scatter(np.real(ep1_obs), np.imag(ep1_obs), marker="+", c='r', s=200)
# g.axes[1,1].axvline(np.imag(ep1_obs), color='r', linestyle='solid', linewidth=0.5)
g.savefig('outputs/Z2/1capa/bayes-'+name+'.png')

print()
print('input ep1_r: ', np.real(ep1_obs))
print('output ep1_r: ', str(ep1_r_mean)+" +/- "+str(ep1_r_std))
print()
print('input ep1_i: ', round(np.imag(ep1_obs),3))
print('output ep1_i: ', str(ep1_i_mean)+" +/- "+str(ep1_i_std))
print()

t3 = time.time()
print('Tiempo insumido - Graficos: '+str(round(t3-t2,2))+' seg', '('+str(round((t3-t2)/60,2))+' min)')
print()

print('Tiempo insumido - TOTAL: '+str(round(t3-t0,2))+' seg', '('+str(round((t3-t0)/60,2))+' min)')
print()
print()
print()

#-----------------------------
EP1 = ep1_r_mean + 1j*ep1_i_mean
S_fit = Scal*RR_ep(EP1,theta)

plt.figure(figsize=(12,8))
plt.plot(f_pos/1e9,np.abs(Star),'-.',label="Medicion")
plt.plot(f_pos/1e9,np.abs(Scal),label="Calibracion")
plt.plot(f_pos/1e9,np.abs(S_fit),label="Ajuste")
plt.grid()
plt.xlabel('Frec [GHz]')
plt.ylabel('Amplitud')
plt.title('Mediciones en valor absoluto')
plt.legend(loc="best")
plt.savefig('outputs/Z2/1capa/fig-'+name+'.png')

#----------------------------------

with open('outputs/Z2/1capa/'+name+'.txt', 'w') as f:
    print('-------------------', file=f)
    print('Incidencia en un dielectrico sin rugosidad', file=f)
    print('-------------------', file=f)
    print('', file=f)
    print('Datos:', file=f)
    print('Dia de medicion:', dia, file=f)
    print('angulo:', tita, file=f)
    print('Altura:', altura)
    print('', file=f)
    print('', file=f)
    print('-------------------', file=f)
    print('Corro bayes', file=f)
    print('-------------------', file=f)
    print('Ruido : ',noise_level, file=f)
    print('Muestras : ',muestras, file=f)
    print('', file=f)
    print('Tiempo insumido - Inf. Bayesiana: '+str(round(t1-t0,2))+' seg', '('+str(round((t1-t0)/60,2))+' min)', file=f)
    print('', file=f)
    print('Graficando con un solo dielectrico ...', file=f)
    print('input ep1_r: ', np.real(ep1_obs), file=f)
    print('output ep1_r: ', str(ep1_r_mean)+" +/- "+str(ep1_r_std), file=f)
    print('', file=f)
    print('input ep1_i: ', round(np.imag(ep1_obs),3), file=f)
    print('output ep1_i: ', str(ep1_i_mean)+" +/- "+str(ep1_i_std), file=f)
    print('', file=f)
    print('Tiempo insumido - Graficos: '+str(round(t3-t2,2))+' seg', '('+str(round((t3-t2)/60,2))+' min)', file=f)
    print('Tiempo insumido - TOTAL: '+str(round(t3-t0,2))+' seg', '('+str(round((t3-t0)/60,2))+' min)', file=f)
    print('', file=f)
    print('', file=f)
f.close()


# - Dos dieléctricos

In [ ]:
NLayers = 2
if Y == '0':
    # mayo
    d_obs = 0.12
    
    
elif Y == '1':
    # junio
    d_obs = 0.13
    

elif Y == '2':
    # julio
    d_obs = 0.14

else:
    # agosto
    d_obs = 0.09

## Sin rugosidad

In [ ]:
print('-------------------')
print('Incidencia en un dielectrico con 2 capas sin rugosidad')
print('-------------------')
print()
print('Datos:')
print('Dia de calibracion:', dia_cal)
print('Dia de target:', dia_tar)
print('Altura:', altura)
name = 'lisa-C'+X+'-T'+Y+'-A'+altura
print()
t0 = time.time()


print('-------------------')
print('Corro bayes')
print('-------------------')
# nro de muestras que va a utilizar el método
muestras = 5000
noise_level = 2.25e-2
# error de la medición
sigmas = noise_level*np.ones((1,len(Star)))

print('Ruido : ',noise_level)
print('Muestras : ',muestras)
print()

# inferencia bayesiana
traza_sim_2layers = modelo(Star,muestras,sigmas,NLayers,rug=False)

t1 = time.time()
print('Tiempo insumido - Inf. Bayesiana: '+str(round(t1-t0,2))+' seg', '('+str(round((t1-t0)/60,2))+' min)')

#Ploteo
t2 = time.time()
print('Graficando para 2 layers ...')
print()

ep1_r_mean = round(np.mean(traza_sim_2layers['ep1_r']),3)
ep1_r_std = round(np.std(traza_sim_2layers['ep1_r']),3)

ep1_i_mean = round(np.mean(traza_sim_2layers['ep1_i']),3)
ep1_i_std = round(np.std(traza_sim_2layers['ep1_i']),3)

ep2_r_mean = round(np.mean(traza_sim_2layers['ep2_r']),3)
ep2_r_std = round(np.std(traza_sim_2layers['ep2_r']),3)

ep2_i_mean = round(np.mean(traza_sim_2layers['ep2_i']),3)
ep2_i_std = round(np.std(traza_sim_2layers['ep2_i']),3)

d_mean = round(np.mean(traza_sim_2layers['d']),3)
d_std = round(np.std(traza_sim_2layers['d']),3)


plotear = np.stack((traza_sim_2layers['ep1_r'], traza_sim_2layers['ep1_i'], traza_sim_2layers['ep2_r'], 
                    traza_sim_2layers['ep2_i'], traza_sim_2layers['d'])).T 
dfplotear = pd.DataFrame(plotear, columns = ['ep1_r','ep1_i','ep2_r','ep2_i','d'])

g = sns.PairGrid(dfplotear)
# g.fig.suptitle(zonaMed, y=1.01, fontsize = 'xx-large')
g.map_upper(sns.histplot)
#g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True, color='.5')
g.savefig('outputs/S2/2capas/bayes-'+name+'.png')

# g.axes[0,0].axvline(np.real(ep1_s), color='r', linestyle='solid', linewidth=0.5)
# g.axes[1,1].axvline(np.imag(ep1_s), color='r', linestyle='solid', linewidth=0.5)
# g.axes[2,2].axvline(np.real(ep2_s), color='r', linestyle='solid', linewidth=0.5)
# g.axes[3,3].axvline(np.imag(ep2_s), color='r', linestyle='solid', linewidth=0.5)
# g.axes[4,4].axvline(d_s, color='r', linestyle='solid', linewidth=0.5)

# g.axes[1,0].scatter(np.real(ep1_s), np.imag(ep1_s), marker="+", c='r', s=200)
# g.axes[2,0].scatter(np.real(ep1_s), np.real(ep2_s), marker="+", c='r', s=200)
# g.axes[3,0].scatter(np.real(ep1_s), np.imag(ep2_s), marker="+", c='r', s=200)
# g.axes[4,0].scatter(np.real(ep1_s), d_s, marker="+", c='r', s=200)

# g.axes[2,1].scatter(np.imag(ep1_s), np.real(ep2_s), marker="+", c='r', s=200)
# g.axes[3,1].scatter(np.imag(ep1_s), np.imag(ep2_s), marker="+", c='r', s=200)
# g.axes[4,1].scatter(np.imag(ep1_s), d_s, marker="+", c='r', s=200)

# g.axes[3,2].scatter(np.real(ep2_s), np.imag(ep2_s), marker="+", c='r', s=200)
# g.axes[4,2].scatter(np.real(ep2_s), d_s, marker="+", c='r', s=200)

# g.axes[4,3].scatter(np.imag(ep2_s), d_s, marker="+", c='r', s=200)


print()
print('input ep1_r: ', np.real(ep1_obs))
print('output ep1_r: ', str(ep1_r_mean)+" +/- "+str(ep1_r_std))
print()
print('input ep1_i: ', np.imag(ep1_obs))
print('output ep1_i: ', str(ep1_i_mean)+" +/- "+str(ep1_i_std))
print()
print('input ep2_r: ', np.real(ep2_obs))
print('output ep2_r: ', str(ep2_r_mean)+" +/- "+str(ep2_r_std))
print()
print('input ep2_i: ', np.imag(ep2_obs))
print('output ep2_i: ', str(ep2_i_mean)+" +/- "+str(ep2_i_std))
print()
print('input d: ', d_obs)
print('output d: ', str(d_mean)+" +/- "+str(d_std))
print()

t3 = time.time()
print('Tiempo insumido - Graficos: '+str(round(t3-t2,2))+' seg', '('+str(round((t3-t2)/60,2))+' min)')
print()

print('Tiempo insumido - TOTAL: '+str(round(t3-t0,2))+' seg', '('+str(round((t3-t0)/60,2))+' min)')
print()
print()
print()

#---------------------------------------------
EP1 = ep1_r_mean + 1j*ep1_i_mean
EP2 = ep2_r_mean + 1j*ep2_i_mean
Dis = d_mean

S_fit = Scal*R_ep1_ep2(EP1,EP2,Dis,f_pos)

plt.figure(figsize=(12,8))
plt.plot(f_pos/1e9,np.abs(Star),'-.',label="Medicion")
plt.plot(f_pos/1e9,np.abs(Scal),label="Calibracion")
plt.plot(f_pos/1e9,np.abs(S_fit),label="Ajuste")
plt.grid()
plt.xlabel('Frec [GHz]')
plt.ylabel('Amplitud')
plt.title('Mediciones en valor absoluto')
plt.legend(loc="best")
plt.savefig('outputs/S2/2capas/fig-'+name+'.png')
#----------------------------------------------

with open('outputs/S2/2capas/'+name+'.txt', 'w') as f:
    print('-------------------', file=f)
    print('Incidencia en un dielectrico con 2 capas sin rugosidad', file=f)
    print('-------------------', file=f)
    print('', file=f)
    print('Datos:', file=f)
    print('Dia de calibracion:', dia_cal, file=f)
    print('Dia de target:', dia_tar, file=f)
    print('Altura:', altura, file=f)
    print('', file=f)
    print('', file=f)

    print('-------------------', file=f)
    print('Corro bayes', file=f)
    print('-------------------', file=f)
    print('Ruido : ',noise_level, file=f)
    print('Muestras : ',muestras, file=f)
    print('', file=f)
    print('Tiempo insumido - Inf. Bayesiana: '+str(round(t1-t0,2))+' seg', '('+str(round((t1-t0)/60,2))+' min)', file=f)
    print('', file=f)

    print('Graficando con un solo dielectrico ...', file=f)
    print('input ep1_r: ', np.real(ep1_obs), file=f)
    print('output ep1_r: ', str(ep1_r_mean)+" +/- "+str(ep1_r_std), file=f)
    print('', file=f)
    print('input ep1_i: ', round(np.imag(ep1_obs)),3, file=f)
    print('output ep1_i: ', str(ep1_i_mean)+" +/- "+str(ep1_i_std), file=f)
    print('', file=f)
    print('input ep2_r: ', np.real(ep2_obs), file=f)
    print('output ep2_r: ', str(ep2_r_mean)+" +/- "+str(ep2_r_std), file=f)
    print('', file=f)
    print('input ep2_i: ', np.imag(ep2_obs), file=f)
    print('output ep2_i: ', str(ep2_i_mean)+" +/- "+str(ep2_i_std), file=f)
    print('', file=f)
    print('input d: ', d_obs, file=f)
    print('output d: ', str(d_mean)+" +/- "+str(d_std), file=f)
    print('', file=f)
    print('Tiempo insumido - Graficos: '+str(round(t3-t2,2))+' seg', '('+str(round((t3-t2)/60,2))+' min)', file=f)
    print('Tiempo insumido - TOTAL: '+str(round(t3-t0,2))+' seg', '('+str(round((t3-t0)/60,2))+' min)', file=f)
    print('', file=f)
    print('', file=f)
f.close()